<a href="https://colab.research.google.com/github/JuanArmas/Proyecto_Final_CEIABD/blob/main/Tratamiento_Datos_2daParte_Version_I_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Juan Airam Armas Alemán

Github: https://github.com/JuanArmas/Proyecto_Final_CEIABD/blob/main/Tratamiento_Datos_2daParte_Version_I_ipynb.ipynb

Drive:
https://drive.google.com/drive/folders/1FvCMo_QYz68XXPFFX4b4TJPTSollnmgB?usp=sharing


In [1]:
import numpy as np
import pandas as pd

from google.colab import drive
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


#Carga de Datos

- Cargamos los datos previamente filtrados y almacenados en el drive.

- A su vez se carga el dataset de festivos y laborales que tenemos alojado también en drive.

In [23]:
# Cargar los datos
nombre_aparcamiento = input("Introduce el nombre del aparcamiento: ")
nombre_aparcamiento = nombre_aparcamiento.upper()
df = pd.read_csv(f"/content/drive/MyDrive/Proyecto_Final/Datasets_creados/{nombre_aparcamiento}.csv")
df_festivo_laboral = pd.read_csv('/content/drive/MyDrive/Proyecto_Final/Datasets_creados/dias_2023.csv')

Introduce el nombre del aparcamiento: rincón


Comprobaciones de que todo va como debiera y eliminación de columna "Estado" que servía de guía.

In [24]:
filas_df_festivo_laboral = len(df_festivo_laboral)
filas_df_festivo_laboral

365

In [25]:
df_festivo_laboral.columns

Index(['Fecha', 'Estado', 'Valor'], dtype='object')

In [26]:
df_festivo_laboral.drop('Estado', axis=1, inplace=True)
df_festivo_laboral.columns

Index(['Fecha', 'Valor'], dtype='object')

# Inicio de la normalización de los datasets
- Se trabajan las fechas para tenerlas en un formato fácil de recorrer. El formato será año, mes y día sin separadores. ej: 20230101 para el 1 de Enero del 2023.
- Nos aseguramos de que las fechas son enteros y las redondeamos en última instancia.

In [27]:
# Convertimos la columna 'EventTime' a datetime; ISO8601”-> Analizar cualquier cadena de tiempo
df['EventTime'] = pd.to_datetime(df['EventTime'], format='ISO8601')

# Extraemos la fecha y la hora como variables separadas
df['fecha'] = df['EventTime'].dt.date
df['hora'] = df['EventTime'].dt.time

# Convertimos la columna "fecha" a string con el formato deseado
fecha_str = df['fecha'].astype(str)

# Convertimos la columna "hora" a string con el formato deseado y redondeamos siempre
hora_str = df['EventTime'].dt.strftime('%H')

# Redondeamos las horas
hora_str = [f'{round(int(x))}' for x in hora_str]

# Convertimos los strings a enteros
fecha_int = [int(x.replace('-', '')) for x in fecha_str]
hora_int = [int(x) for x in hora_str]

- Se separan las horas de la fecha y el día
- Se reemplazan los valores categóricos por numéricos, en este caso se hace manualmente al ser pocos los cambios a realizar, entrada/salida y nombres de aparcamientos (1-7)
- Se reordenan las columnas del dataset para su mejor interpretación.

In [28]:
del df['EventTime']
df = df.reindex(['APARCAMIENTO','fecha','hora','EventDesc'], axis=1)

# Introducimos la fecha y hora numérica a la tabla
df['fecha'] = fecha_int
df['hora'] = hora_int

# Reemplazamos los valores categóricos por enteros
df = df.replace({"ENTRADA": 0, "SALIDA": 1, 'Entrada': 0, 'Salida': 1,'RINCÓN': 1, 'ELDER': 2, 'SAN BERNARDO': 3, 'SANAPÚ': 4, 'MATA': 5, 'VEGUETA': 6, 'METROPOL': 7})

- Se crean dos columnas nuevas entrada/salida, extrayendo de la columna eventdesc el numero de entradsa y salidas por día. Se borra la columna eventdesc.

In [29]:
# Dividir los valores de EventDesc en dos columnas nuevas: Entrada y Salida
df['Entrada'] = df['EventDesc'].apply(lambda x: 1 if x == 1 else 0)
df['Salida'] = df['EventDesc'].apply(lambda x: 1 if x == 0 else 0)

# Eliminar la columna EventDesc si ya no se necesita
df.drop('EventDesc', axis=1, inplace=True)

print(f"Mostrando datos de {nombre_aparcamiento}:")
df


Mostrando datos de RINCÓN:


,APARCAMIENTO,fecha,hora,Entrada,Salida
0,1,20230101,0,0,1
1,1,20230101,0,1,0
2,1,20230101,0,0,1
3,1,20230101,0,0,1
4,1,20230101,0,0,1
...,...,...,...,...,...
1036481,1,20231230,0,1,0
1036482,1,20231230,12,0,1
1036483,1,20231230,16,1,0
1036484,1,20231231,11,0,1


In [31]:
df.to_csv(f'dividido_Ent_sal_{nombre_aparcamiento}.csv', index=False)

- Se agrupan por fecha y hora las entradas y salidas para tener los totales en las 24h

In [32]:
df_agrupado = df.groupby(['APARCAMIENTO', 'fecha', 'hora']).sum().reset_index()

print(df_agrupado)

      APARCAMIENTO     fecha  hora  Entrada  Salida
0                1  20230101     0      102      86
1                1  20230101     1      117     189
2                1  20230101     2       73      88
3                1  20230101     3       40      13
4                1  20230101     4       29       5
...            ...       ...   ...      ...     ...
8669             1  20231231    19      142      31
8670             1  20231231    20      121      44
8671             1  20231231    21       31      58
8672             1  20231231    22       17      51
8673             1  20231231    23       13     129

[8674 rows x 5 columns]


In [33]:
df_agrupado.to_csv(f'Agrupado_{nombre_aparcamiento}.csv', index=False)

In [ ]:
# Dividimos los datos en características (X) y etiquetas (y)
X = df[['fecha', 'hora']]
y = df['EventDesc']

In [ ]:
# # Escalamos los datos
# scaler = StandardScaler()
# X_normalized = scaler.fit_transform(X)

X_values = X.values()

In [ ]:
# Dividimos los datos en conjuntos de entrenamiento y prueba (80% train, 20% test)
# X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_values, y, test_size=0.2, random_state=42)

# Dividir los datos de entrenamiento nuevamente para obtener un conjunto de validación (60% train, 20% validation, 20% test)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# El 25% de los datos de entrenamiento se convierten en validación